In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd

def get_data():
    url = "s3://wagon-public-datasets/taxi-fare-train.csv"
    df = pd.read_csv(url, nrows=100)
    return df

def clean_df(df):
    df = df.dropna(how='any', axis='rows')
    df = df[(df.dropoff_latitude != 0) | (df.dropoff_longitude != 0)]
    df = df[(df.pickup_latitude != 0) | (df.pickup_longitude != 0)]
    if "fare_amount" in list(df):
        df = df[df.fare_amount.between(0, 4000)]
    df = df[df.passenger_count < 8]
    df = df[df.passenger_count >= 0]
    df = df[df["pickup_latitude"].between(left=40, right=42)]
    df = df[df["pickup_longitude"].between(left=-74.3, right=-72.9)]
    df = df[df["dropoff_latitude"].between(left=40, right=42)]
    df = df[df["dropoff_longitude"].between(left=-74, right=-72.9)]
    return df

In [ ]:
df = get_data()

In [ ]:
df = clean_df(df)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model_params = dict(
  n_estimators=100,
  max_depth=1)

model = RandomForestRegressor()
model.set_params(**model_params)

In [ ]:
from sklearn.model_selection import train_test_split

y_train = df["fare_amount"]
X_train = df.drop("fare_amount", axis=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1)

In [ ]:
def minkowski_distance(df, p,
                       start_lat="pickup_latitude",
                       start_lon="pickup_longitude",
                       end_lat="dropoff_latitude",
                       end_lon="dropoff_longitude"):
    x1 = df[start_lon]
    x2 = df[end_lon]
    y1 = df[start_lat]
    y2 = df[end_lat]
    return ((abs(x2 - x1) ** p) + (abs(y2 - y1)) ** p) ** (1 / p)

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class DistanceTransformer(BaseEstimator, TransformerMixin):

    def __init__(self, distance_type="euclidian", **kwargs):
        self.distance_type = distance_type

    def transform(self, X, y=None):
        assert isinstance(X, pd.DataFrame)
        if self.distance_type == "haversine":
            X["distance"] = haversine_vectorized(X)
        if self.distance_type == "euclidian":
            X["distance"] = minkowski_distance(X, p=2)
        if self.distance_type == "manhattan":
            X["distance"] = minkowski_distance(X, p=1)
        return X[["distance"]]

    def fit(self, X, y=None):
        return self

In [ ]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

pipe_distance = make_pipeline(
    DistanceTransformer(),
    StandardScaler())


cols = ["pickup_latitude", "pickup_longitude", "dropoff_latitude", "dropoff_longitude"]

feateng_blocks = [
    ('distance', pipe_distance, cols),
]

features_encoder = ColumnTransformer(feateng_blocks)

pipeline = Pipeline(steps=[
            ('features', features_encoder),
            ('model', model)])

In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
def compute_rmse(y_pred, y_true):
    return np.sqrt(((y_pred - y_true) ** 2).mean())

In [ ]:
y_pred = pipeline.predict(X_val)

In [ ]:
rmse = compute_rmse(y_pred, y_val)

In [ ]:
rmse

In [ ]:
from memoized_property import memoized_property

import mlflow
from  mlflow.tracking import MlflowClient

class MLFlowBase():

    def __init__(self, experiment_name, MLFLOW_URI):
        self.experiment_name = experiment_name
        self.MLFLOW_URI = MLFLOW_URI

    @memoized_property
    def mlflow_client(self):
        mlflow.set_tracking_uri(self.MLFLOW_URI)
        return MlflowClient()

    @memoized_property
    def mlflow_experiment_id(self):
        try:
            return self.mlflow_client \
                .create_experiment(self.experiment_name)
        except BaseException:
            return self.mlflow_client \
                .get_experiment_by_name(self.experiment_name).experiment_id

    def mlflow_create_run(self):
        self.mlflow_run = self.mlflow_client \
            .create_run(self.mlflow_experiment_id)

    def mlflow_log_param(self, key, value):
        self.mlflow_client \
            .log_param(self.mlflow_run.info.run_id, key, value)

    def mlflow_log_metric(self, key, value):
        self.mlflow_client \
            .log_metric(self.mlflow_run.info.run_id, key, value)


In [ ]:
pipe.get_params()

In [ ]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(
    pipe, 
    param_grid={
        'feat_eng_bloc__distance__robustscaler__copy': [True],
        'regressor__min_samples_leaf': [3],
        'regressor__oob_score': [True],
        'regressor__min_weight_fraction_leaf': [0.0, 0.1]
    },
    cv=5
)

grid_search.fit(X_train, y_train)
grid_search.score(X_test, y_test)

grid_search.best_estimator_
grid_search.best_params_